# Density computational times
The research question for this notebook is: "What is a nice resolution to calculate the density?"

This depends on the computational time, and the accountability of the outcome.

In [ ]:
import sys

sys.path.append('..//helpers//')
sys.path.append('..//classes//')
sys.path.append('..//constants//')
sys.path.append('../')

In [ ]:
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os
import time
import numpy as np
import csv
import pandas as pd

from calc_density_4 import count_points_per_square

from Settings import Settings

from geometry_helpers import make_coordinate_df
from density_helpers import prepare_df

from paths import WORKDIR, RESULTSDIR, RADII_CSV

In [ ]:
central_groups = ["RCOMe", "RNO2", "ArCI", "NO3", "RC6F5", "H2O", "RC6H5"]
contact_groups = ["CF", "RCN", "R2CO", "XH", "XH", "CCH3", "C2CH2", "RC6H5", "ArCH"]  #
to_count =       ["F",   "N",    "O",   "H", "O",  "H",     "H", "centroid", "H"] #, 

central_groups = ["ArCI"]
contact_groups = ["ArCH"]  #
to_count =       ["H"] #, 

resolutions = np.arange(0.1, 1.6, 0.1)

resolutions = [0.25]

resolutions = np.flip(resolutions)

# Data Gathering

In [ ]:
rerun = True
write_time = False

if rerun:
    with open('../results/density_comp_time.csv', 'a', newline='') as resultsfile:
        if write_time:
            writer = csv.writer(resultsfile)
            writer.writerow(['central', 'contact', 'to_count', 'resolution', 'density_time'])

        for central_group in central_groups:
            for to_count_contact, contact_group in zip(to_count, contact_groups):
                datafile = ".\\data\\" + central_group + "\\" + central_group + "_" + contact_group + "_vdw.5.cor"
                result1 = ".\\results\\" + central_group + "\\" + central_group + "_" + contact_group + "_vdw.5\\"\
                          + central_group + "_" + contact_group + "_kabsch_aligned.csv"

                for resolution in resolutions:
                    print("\nCalculating density for central group: ", central_group, " contact group: ", contact_group,
                          "resolution: ", str(round(resolution, 2)))

                    t0 = time.time()

                    settings = Settings(RESULTSDIR, RADII_CSV, datafile)
                    settings.set_atom_to_count(to_count_contact)

                    # resolution of the bins, in Angstrom
                    settings.set_resolution(round(resolution, 2))
                    
                    df = pd.read_csv(settings.get_kabsch_aligned_csv_filename())
                    avg_frag = pd.read_csv(settings.get_avg_frag_filename())

                    # grab only the atoms that are in the contact groups
                    df_central = df[df['label'] == '-']
                    coordinate_df = make_coordinate_df(df_central, settings, avg_frag)

                    empty_density_df = prepare_df(df=coordinate_df, settings=settings)

                    density_df = count_points_per_square(df=empty_density_df, contact_points_df=coordinate_df, settings=settings)

                    # save so we can use the data but only change the plot - saves time :)
                    density_df.to_hdf(settings.get_density_df_filename(), settings.get_density_df_key())               

                    t1 = time.time() - t0
                    print("Duration: %.2f s." % t1)      
                    
                    if write_time:
                        writer.writerow([central_group, contact_group, to_count_contact, round(resolution, 2), t1])

# Analyzing data

In [ ]:
df = pd.read_csv('../results/density_comp_time.csv')

for central in central_groups:
    plt.figure(figsize=(6,4))
    for contact, to_count_contact in zip(contact_groups, to_count):
        dfje = df[(df.central == central) & (df.contact == contact)]
        print(len(dfje))
        plt.plot(dfje.resolution, dfje.density_time, label=central + '-' + contact)
        plt.scatter(dfje.resolution, dfje.density_time)
    
    plt.legend()
    plt.xlabel("Resolution")
    plt.ylabel("Time (s)")
    plt.title("Density computational time " + central)
    plt.savefig("results/figures/comp_time_" + central + ".svg", format='svg')

# Merge results with pre comp results

In [ ]:
display(df)
grouped = df.groupby(['central', 'contact', 'to_count'])

In [ ]:
id_df = grouped['density_time'].apply(lambda x: pd.Series(x.values)).unstack()
id_df = id_df.reset_index()

In [ ]:
id_df.columns = ['central', 'contact', 'to_count', 'res15', 'res14', 'res13', 'res12', 'res11',\
                                                   'res10', 'res09', 'res08', 'res07', 'res06',\
                                                   'res05', 'res04', 'res03', 'res02', 'res01']
display(id_df)

In [ ]:
df_pre = pd.read_csv('results/pre_density_comp_time.csv')

df_total = pd.merge(df_pre, id_df, how='left', left_on=['contact','central', 'to_count'], right_on = ['contact','central', 'to_count'])
display(df_total)

In [ ]:
i = 0
r1 = []
xtick_labels = []

fig, ax = plt.subplots(figsize=(9,15))
    
for _, row in df_total.iterrows():
    bars1 = row['alignment']
    bars2 = row['avg_fragment']
    bars3 = row['coordinate_df']
    bars4 = row['res01']
    bars5 = row['res02']
    bars6 = row['res03']
    bars7 = row['res04']
    bars8 = row['res05']
        
    bar_width = 0.5  # the width of the bars
    r1.append(i)

    bottom = 0
    # make that plot
    rects1 = plt.bar(i, bars1, bar_width, bottom=0, color="tab:blue")
    
    bottom += bars1
    rects2 = plt.bar(i, bars2, bar_width, bottom=bottom, color="tab:orange")
    
    bottom += bars2
    rects3 = plt.bar(i, bars3, bar_width, bottom=bottom, color="tab:green")
    
    bottom += bars3
    rects01 = plt.bar(i, bars4, bar_width, bottom=bottom, color='tab:red')
    
    bottom += bars4
    rects02 = plt.bar(i, bars5, bar_width, bottom=bottom, color='tab:purple')
    
    bottom += bars5
    rects03 = plt.bar(i, bars6, bar_width, bottom=bottom, color='tab:brown')
    
    bottom += bars6
    rects04 = plt.bar(i, bars7, bar_width, bottom=bottom, color='tab:pink')
    
    bottom += bars7
    rects05 = plt.bar(i, bars8, bar_width, bottom=bottom, color='tab:gray')
    
    xtick_labels.append(row["central"] + "-" + row['contact'])
    i+=1


    
plt.xticks(r1, xtick_labels, rotation=90)

plt.title('Computational times')
plt.subplots_adjust(bottom=0.3)

ax.set_xlabel("Pair")
ax.set_ylabel("Computational time (s)")

# ax.set_ylim(0,100)

ax2 = ax.twinx()
ax2.set_ylabel("Amount")
# ax2.set_ylim(0, 600000)

line = ax2.plot(range(len(df_total)), df_total["amount_structures"], color="red", label="no unique fragments")

plt.legend((rects1[0], rects2[0], rects3[0], rects01[0], rects02[0], rects03[0], rects04[0], rects05[0], line[0]),
           ('alignment', 'avg_fragment', 'coordinate_df', 'density res 0.1', 'density res 0.2',\
            'density res 0.3', 'density res 0.4', 'density res 0.5', 'No. fragments'))

plt.savefig("results/figures/comp_times_total.svg", format="svg")

plt.show()

In [ ]:
df_total.describe()

In [ ]:
# df_total = df_total.drop(columns=['Unnamed: 0', 'total'])
print(df_total.columns)

In [ ]:
# df_total = df_total[['central', 'contact', 'to_count', 'alignment', 'coordinate_df',
#        'avg_fragment', 'total', 'res15', 'res14', 'res13', 'res12', 'res11', 'res10', 'res09', 'res08',
#        'res07', 'res06', 'res05', 'res04', 'res03', 'res02', 'res01']]

# sort again
df_total['total_density'] = df_total.iloc[:, -15:].sum(axis=1)
df_total['total'] = df_total['total_density'] + df_total['alignment'] + df_total['avg_fragment'] + df_total['coordinate_df']
df_total = df_total.sort_values("total", ascending=False)

In [ ]:
i = 0
r1 = []
xtick_labels = []

fig, ax = plt.subplots(figsize=(9,12))
    
for _, row in df_total.iterrows():
    bars1 = row['alignment']
    bars2 = row['avg_fragment']
    bars3 = row['coordinate_df']
    bars4 = row['res01']
    bars5 = row['res02']
    bars6 = row['res03']
    bars7 = row['res04']
    bars8 = row['res05']
        
    bar_width = 0.5  # the width of the bars
    r1.append(i)

    bottom = 0
    # make that plot
    rects1 = plt.bar(i, bars1, bar_width, bottom=0, color="tab:blue")
    
    bottom += bars1
    rects2 = plt.bar(i, bars2, bar_width, bottom=bottom, color="tab:orange")
    
    bottom += bars2
    rects3 = plt.bar(i, bars3, bar_width, bottom=bottom, color="tab:green")
    
    bottom += bars3
    rects01 = plt.bar(i, bars4, bar_width, bottom=bottom, color='tab:red')
    
    bottom += bars4
    rects02 = plt.bar(i, bars5, bar_width, bottom=bottom, color='tab:purple')
    
    bottom += bars5
    rects03 = plt.bar(i, bars6, bar_width, bottom=bottom, color='tab:brown')
    
    bottom += bars6
    rects04 = plt.bar(i, bars7, bar_width, bottom=bottom, color='tab:pink')
    
    bottom += bars7
    rects05 = plt.bar(i, bars8, bar_width, bottom=bottom, color='tab:gray')
    
    xtick_labels.append(row["central"] + "-" + row['contact'])
    i+=1


    
plt.xticks(r1, xtick_labels, rotation=90)

plt.title('Computational times')
plt.subplots_adjust(bottom=0.3)

ax.set_xlabel("Pair")
ax.set_ylabel("Computational time (s)")

# ax.set_ylim(0,100)

ax2 = ax.twinx()
ax2.set_ylabel("Amount")
# ax2.set_ylim(0, 600000)

line = ax2.plot(range(len(df_total)), df_total["amount_structures"], color="gold", label="no unique fragments")

plt.legend((rects1[0], rects2[0], rects3[0], rects01[0], rects02[0], rects03[0], rects04[0], rects05[0], line[0]),
           ('alignment', 'avg_fragment', 'coordinate_df', 'density res 0.1', 'density res 0.2',\
            'density res 0.3', 'density res 0.4', 'density res 0.5', 'No. fragments'))

plt.savefig("results/figures/comp_times_total.svg", format="svg", bbox_inches='tight')
plt.savefig("results/figures/comp_times_total.png")

plt.show()

In [ ]:
df_total.describe()

In [ ]:
df_test = df_total.copy()
df_test['total'] = df_test['total'] - df_test['res01']
df_test = df_test.drop(columns=['res01'])
df_test.describe()

# Conclusions